In [5]:
import os 
import pandas as pd 
import numpy as np 
import mne 
from scipy.interpolate import interp1d

from typing import Optional

import seaborn as sns 

In [6]:
def is_integer_with_custom_precision(num: float, eps: float = 1e-6, int_value: Optional[int] = None):
    if int_value is None:
        int_value = int(num)
    return abs(int_value - num) < eps, int_value



In [7]:
EEG_SAMPLING_RATE = 250
FMRI_TR = 2.0  # sec

RUNS = [
    ('rs', '01'),
    ('rs', '02'),
    ('rs', '03'),
    ('rs', '04'),
    ('rs', '05'),
    ('rs', '06'),
    ('socword', '01'), 
    ('socword', '02'),
    ('socword', '03'),
    ('socword', '04'),
    ('socword', '05'),
    ('socword', '06')
    ]
RUNSNATVIEW = [
    ('01', 'checker'),
    ('01', 'dme_run-01'),
    ('01', 'dme_run-02'),
    ('01', 'inscapes'),
    ('01', 'monkey1_run-01'),
    ('01', 'monkey1_run-02'),
    ('01', 'rest'),
    ('01', 'tp_run-01'),
    ('01', 'tp_run-02')
]

RUNS_1895 = [
    ('rs', '01'),
    ('rs', '02'),
    ('socword', '01'), 
    ('socword', '02'),
    ('socword', '03')
    ]
# IN SAMPLES WITH 2 Hz SAMPLING RATE. FOR SECONDS DIVIDE BY 2
TEST_SIZES = {
    'checker': 100,
    'dme': 250,
    'inscapes': 250,
    'monkey1': 150,
    'rest': 250,
    'tp': 100,
    'dmh': 250,
    'monkey2': 150,
    'monkey5': 150
}


In [8]:

def load_natdataset_eeg_data(root: str, sub: str, ses: str, task: str, eeg_channels: Optional[list[str]]):
    """
    Loads EEG data, checks it and crops by first fMRI scan

    Parameters
    ----------
    root : str
        A directory where the dataset is
    sub : str
        A subject id
    ses : str
        A session id
    task : str
        A task id
    eeg_channels : list[str] or None
        A list of EEG channels to load from file. If None, extracts every channel available

    Returns
    -------
    df_eeg : pd.DataFrame
        A DataFrame with EEG data. Columns are EEG channels
    eeg_channels : list[str]
        A list of EEG channels loaded from file
    vhdr_path : str
        A path to the .vhdr file from which the data was extracted
    eeg_times : np.ndarray
        Time-steps of the EEG signal
    """
    filename = f'sub-{sub}/ses-01/eeg/sub-{sub}_ses-{ses}_task-{task}_eegMRbvCBbviiR250.vhdr'
    vhdr_path = os.path.join(root, filename)
    eeg_raw = mne.io.read_raw_brainvision(vhdr_path, preload=True)
    eeg_raw = eeg_raw.pick('eeg')

    events, event_ids = mne.events_from_annotations(eeg_raw)

    r128_scans = events[events[:, -1] == event_ids['Stimulus/R128']]

    unique_scan_diffs, counts_diffs = np.unique(np.diff(r128_scans[:, 0]), return_counts=True)
    assert len(unique_scan_diffs) == 1, (unique_scan_diffs, counts_diffs)
    assert unique_scan_diffs[0] == 525  # 2.1 sec (fMRI TR) * 250 Hz (EEG sampling rate)

    assert eeg_raw.info['sfreq'] == EEG_SAMPLING_RATE
    first_r128 = r128_scans[0, 0] / EEG_SAMPLING_RATE

    cropped_eeg = eeg_raw.crop(tmin=first_r128)

    df_eeg = cropped_eeg.to_data_frame()

    if eeg_channels is None:
        eeg_channels = [column for column in df_eeg.columns if column != 'time']

    eeg_times = df_eeg['time']
    df_eeg = df_eeg[eeg_channels]

    return df_eeg, eeg_channels, vhdr_path, eeg_times, events, event_ids


In [74]:
def load_eeg_data(root: str, ses:str, run:str, task: str, eeg_channels: Optional[list[str]]):

    filename = f'sub-101472_ses-{ses}_task-{task}_run-{run}_eegMRbvCBbviiR250.vhdr'
    eeg_vhdr_path = os.path.join(root, f'ses-{ses}', 'eeg', filename)

    
    eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)
    events, event_ids = mne.events_from_annotations(eeg_raw)
    events_dict_cut = {'Scanner/Scan Start': event_ids['Scanner/Scan Start']}  # use only Scan Start event


    try:
        scan_key = 'Scanner/Scan Start'
        segment_scans = events[events[:, -1] == event_ids[scan_key]]
    except: 
        scan_key = 'Stimulus/Scan Start'
        segment_scans = events[events[:, -1] == event_ids[scan_key]] 
        print(f'ERROR: FAULTY EVENT NAME. PATH:{eeg_vhdr_path}')


    unique_scan_diffs, counts_diffs = np.unique(np.diff(segment_scans[:, 0]), return_counts=True)
    assert len(unique_scan_diffs) == 1, (unique_scan_diffs, counts_diffs)
    assert unique_scan_diffs[0] == 3000  

    assert eeg_raw.info['sfreq'] == EEG_SAMPLING_RATE
    first_segment = segment_scans[0, 0] / EEG_SAMPLING_RATE

    cropped_eeg = eeg_raw.crop(tmin=first_segment)
    df_eeg = cropped_eeg.to_data_frame()

    if eeg_channels is None:
        eeg_channels = [column for column in df_eeg.columns if column != 'time']

    eeg_times = df_eeg['time']


    epochs = mne.Epochs(
        cropped_eeg,
        events = events, 
        event_id =events_dict_cut, 
        tmin = 6.5,
        tmax = 11.5-1/EEG_SAMPLING_RATE, 
        baseline = None, 
        preload = True
    )

    epochs_df = epochs.to_data_frame()
    #try: 
    assert (epochs_df['epoch'].unique() == np.where(events[:,2] == 10002)).all()
    #except: 
        #assert (epochs_df['epoch'].unique() == np.where(events[:,2] == 10001)).all()
        #print(f'FAULTY EVENT NAMES. SESSION:{ses},RUN:{run},  TASK:{task}')
    epochs_array = epochs.get_data()[:-1, ...]
    segment_scans = segment_scans[:-1]


    return epochs_array, eeg_channels, eeg_vhdr_path, eeg_times, events,  event_ids, segment_scans

In [10]:
#UPDATE FUNCTIONS BELOW FOR THE EPOCHS RETURN + ALIGN EPOCHT WITH FMRI  

In [11]:
def load_fmri_data(root, ses, task, run, roi_folder, rois ): 
    """
    Loads pre-processed fMRI data. The original TR is 2.0 
    """
    filename = f'sub-101472_ses-{ses}_task-{task}_run-{run}.csv'
    csv_path = os.path.join(root, f'ses-{ses}', roi_folder, filename)
    fmri_data = pd.read_csv(csv_path)

    if rois is None:
        rois = [column for column in fmri_data.columns if column not in ['Nvol', ' Tstart']]
    # assert for tsart integer 
    assert all([i.is_integer() for i in fmri_data[' Tstart']])
    fmri_times = fmri_data[' Tstart'].to_numpy()

    fmri_times = fmri_times.astype(int)
    fmri_times = fmri_times//12

    fmri_data = fmri_data.assign(N_cycle=pd.Series(fmri_times).values)
    df_ncycle = fmri_data.groupby(by = 'N_cycle')

    fmri_array = np.full((len(df_ncycle), len(rois), 3), fill_value = np.inf)

    for idx, group in df_ncycle: 
        group = group[rois].to_numpy()
        fmri_array[idx, ...] = group.T 

    # check if no inf that were previously filled are left  
    assert not np.isinf(fmri_array).any(), (f'') 

    # cut the first triplet 
    fmri_array = fmri_array[1:, ...]

    # if averaging for 3 scans; fmri_array = np.mean(fmri_array, axis = -1)
    # better move to pytorch dataset 

    return fmri_array, fmri_times, rois, csv_path, fmri_data



In [52]:
def load_data(root: str, run: str, task: str, ses:str,
              rois: Optional[list[str]], eeg_channels: Optional[list[str]],
              sampling_rates_ratio = 500, roi_folder: str = 'roi'):
    
    fmri_array, fmri_times, rois, fmri_path, fmri_data = load_fmri_data(
        root=root, run=run, ses=ses, task=task, rois=rois, roi_folder=roi_folder
    )

    epochs_array, eeg_channels, eeg_path, eeg_times, events, event_ids, segment_scans = load_eeg_data(
        root=root, ses=ses, run=run, task=task, eeg_channels=eeg_channels
    )

    assert (fmri_array.shape[0]) == len(segment_scans), (f'EEG segment marks should be 3x '
                                                 f'fMRI sample size. '
                                                 f'fMRI sample size is {(fmri_array.shape[0])}. '
                                                 f'Number of segment marks: {len(segment_scans)} '
                                                 f'file is Root: {root},  ses: {ses}, run: {run},'
                                                 f' task: {task}.')
    


   

    # assert epochs.shape[0] == (df_fmri.shape[0] * sampling_rates_ratio), (f'EEG size: {df_eeg.shape}, fMRI '
    #                                                                       f'size: {df_fmri.shape}, sr ratio: '
    #                                                                       f'{sampling_rates_ratio}.\nRoot: {root},  '
    #                                                                       f'sub: {sub}, ses: {ses}, task: {task}.')







    #fmri_for_global_trend = df_fmri[rois_for_global_trend].to_numpy().T



    # center BOLD around 0
    fmri_array = fmri_array - np.mean(fmri_array, axis=1, keepdims=True)
    #fmri_for_global_trend = fmri_for_global_trend - np.mean(fmri_for_global_trend, axis=1, keepdims=True)

    # if separate_global_trend:
    #     assert fmri_array.ndim == 2, fmri_array.shape
    #     assert fmri_for_global_trend.ndim == 2, fmri_for_global_trend.shape
    #     global_trend_array = np.mean(fmri_for_global_trend, axis=0)

    #     inner = np.inner(global_trend_array, global_trend_array)
    #     outer = np.outer(global_trend_array, global_trend_array)

    #     detrend = fmri_array @ outer / inner
    #     fmri_array = fmri_array - detrend

    #     for idx in range(fmri_array.shape[0]):
    #         corr_coeff = np.corrcoef(fmri_array[idx, :], global_trend_array)[0, 1]
    #         assert abs(corr_coeff) < 1e-6, (
    #             f'Sub: {sub}, ses: {ses}, task: {task}, ROI: {rois[idx]} has correlation above threshold: {corr_coeff}'
    #         )

    #     fmri_array = np.concatenate((fmri_array, global_trend_array[None, :]), axis=0)
    #     assert np.equal(fmri_array[-1], global_trend_array).all()

    # fix so that the return for eeg are epochs 
    return epochs_array, fmri_array, eeg_channels, rois, eeg_path, fmri_path, eeg_times, events, event_ids, segment_scans


In [13]:

# 3. check last event for xtra tail 


In [80]:
def load_multiple_files(root: str,
                        rois: Optional[list[str]], eeg_channels: Optional[list[str]], sessions:list, starting_point_sec: int = 0,
                        rois_for_global_trend: Optional[list[str]] = None, roi_folder: str = 'roi'):
    datasets = {}
    event_ids_dict = {}
    if rois is None:
        overall_rois = None
    else:
        overall_rois = list(rois)

    if eeg_channels is None:
        overall_eeg_channels = None
    else:
        overall_eeg_channels = list(eeg_channels)

    for ses in sessions: 
        if ses == '1895':
            for task, run in RUNS_1895:
                #try:
                    epochs_array, fmri_array, eeg_channels, rois, eeg_path, fmri_path, eeg_times, events, event_ids, segment_scans = load_data(
                        root=root, ses=ses, run=run, task=task, rois=rois,
                        eeg_channels=eeg_channels, sampling_rates_ratio=500,
                        roi_folder=roi_folder
                    )

                    if overall_rois is None:
                        overall_rois = list(rois)
                    else:
                        assert overall_rois == rois

                    if overall_eeg_channels is None:
                        overall_eeg_channels = list(eeg_channels)
                    else:
                        assert overall_eeg_channels == eeg_channels

                    datasets[(eeg_path, fmri_path)] = {
                        'eeg': epochs_array, 'fmri': fmri_array, 'task': task.split('_')[0], 'eeg_times': eeg_times,'events':events, 'event_ids':event_ids,
                        'segment_scans':segment_scans
                    }
                    
                    event_ids_dict[(task, run, ses)] = event_ids
            #except Exception as e:
                #print(e) 
        else:
            for task, run in RUNS:
                #try:
                    epochs_array, fmri_array, eeg_channels, rois, eeg_path, fmri_path, eeg_times, events, event_ids, segment_scans = load_data(
                        root=root, ses=ses, run=run, task=task, rois=rois,
                        eeg_channels=eeg_channels, sampling_rates_ratio=500,
                        roi_folder=roi_folder
                    )

                    if overall_rois is None:
                        overall_rois = list(rois)
                    else:
                        assert overall_rois == rois

                    if overall_eeg_channels is None:
                        overall_eeg_channels = list(eeg_channels)
                    else:
                        assert overall_eeg_channels == eeg_channels

                    datasets[(eeg_path, fmri_path)] = {
                        'eeg': epochs_array, 'fmri': fmri_array, 'task': task.split('_')[0], 'eeg_times': eeg_times,'events':events, 'event_ids':event_ids,
                        'segment_scans':segment_scans
                    }
                    event_ids_dict[(task, run, ses)] = event_ids


    rois = list(rois)
    # if separate_global_trend:
    #     rois.append(' Global Regressive Trend')

    return datasets, rois, eeg_channels, event_ids_dict

In [81]:
root = '/Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472'
sessions = ['1818', '1895']

datasets, rois, eeg_channels, event_ids_dict= load_multiple_files(root = root, sessions=['1818', '1895'],
                                                  rois = None, eeg_channels = None,  starting_point_sec=0,
                                                  rois_for_global_trend = None, roi_folder = 'roi')
     
    

Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/eeg/sub-101472_ses-1818_task-rs_run-01_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138060  =      0.000 ...   552.240 secs...
Used Annotations descriptions: ['New Segment/', 'Pulse Artifact/R', 'Scanner/Scan Start']


/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_75595/2795110670.py:7: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_75595/2795110670.py:7: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)


Not setting metadata
46 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 46 events and 1250 original time points ...
0 bad epochs dropped
Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/eeg/sub-101472_ses-1818_task-rs_run-02_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138059  =      0.000 ...   552.236 secs...
Used Annotations descriptions: ['New Segment/', 'Pulse Artifact/R', 'Scanner/Scan Start']


/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_75595/2795110670.py:7: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_75595/2795110670.py:7: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)


Not setting metadata
46 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 46 events and 1250 original time points ...
0 bad epochs dropped
Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/eeg/sub-101472_ses-1818_task-rs_run-03_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138055  =      0.000 ...   552.220 secs...
Used Annotations descriptions: ['New Segment/', 'Pulse Artifact/R', 'Scanner/Scan Start']


...

Not setting metadata
46 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 46 events and 1250 original time points ...
0 bad epochs dropped
Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1895/eeg/sub-101472_ses-1895_task-socword_run-02_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138057  =      0.000 ...   552.228 secs...


/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_75595/2795110670.py:7: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_75595/2795110670.py:7: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)


Used Annotations descriptions: ['New Segment/', 'Pulse Artifact/R', 'Scanner/Scan Start', 'Stimulus/S  1', 'Stimulus/S  3', 'Stimulus/S  5', 'Stimulus/S  9', 'Stimulus/S 10', 'Stimulus/S 16', 'Stimulus/S 32', 'Stimulus/S 48', 'Stimulus/S 64', 'Stimulus/S 96']
Not setting metadata
46 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 46 events and 1250 original time points ...
0 bad epochs dropped
Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1895/eeg/sub-101472_ses-1895_task-socword_run-03_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138024  =      0.000 ...   552.096 secs...
Used Annotations descriptions: ['New Segment/', 'Pulse Artifact/R', 'Scanner/Scan Start', 'Stimulus/S  1', 'Stimulus/S  3', 'Stimulus/S  5', 'Stimulus/S  9', 'Stimulus/S 10', 'Stimulus/S 32', 'Stimulus/S 48', 'Stimulus/S 64', 'Stimulus/S 80', 'Stimulus/S 96']


/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_75595/2795110670.py:7: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_75595/2795110670.py:7: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)


Not setting metadata
46 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 46 events and 1250 original time points ...
0 bad epochs dropped


In [100]:
for key, value in event_ids_dict.items(): 
    print(value['Scanner/Scan Start'] == 10002)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [101]:
for key, value in event_ids_dict.items():
    if  'socword' in key:
        print(key, value)

('socword', '01', '1818') {'New Segment/': 99999, 'Pulse Artifact/R': 10001, 'Scanner/Scan Start': 10002, 'Stimulus/S  1': 1, 'Stimulus/S  3': 3, 'Stimulus/S  5': 5, 'Stimulus/S  9': 9, 'Stimulus/S 10': 10, 'Stimulus/S 32': 32, 'Stimulus/S 48': 48, 'Stimulus/S 64': 64, 'Stimulus/S 80': 80, 'Stimulus/S 96': 96}
('socword', '02', '1818') {'New Segment/': 99999, 'Pulse Artifact/R': 10001, 'Scanner/Scan Start': 10002, 'Stimulus/S  1': 1, 'Stimulus/S  3': 3, 'Stimulus/S  5': 5, 'Stimulus/S  9': 9, 'Stimulus/S 10': 10, 'Stimulus/S 32': 32, 'Stimulus/S 48': 48, 'Stimulus/S 64': 64, 'Stimulus/S 80': 80, 'Stimulus/S 96': 96}
('socword', '03', '1818') {'New Segment/': 99999, 'Pulse Artifact/R': 10001, 'Scanner/Scan Start': 10002, 'Stimulus/S  1': 1, 'Stimulus/S  3': 3, 'Stimulus/S  5': 5, 'Stimulus/S  9': 9, 'Stimulus/S 10': 10, 'Stimulus/S 16': 16, 'Stimulus/S 32': 32, 'Stimulus/S 48': 48, 'Stimulus/S 64': 64, 'Stimulus/S 80': 80, 'Stimulus/S 96': 96}
('socword', '04', '1818') {'New Segment/': 

In [ ]:

root = '/Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472'
sessions = ['1818', '1895']


In [18]:
len(datasets.keys())

17

In [54]:
datasets[('/Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1804/eeg/sub-101472_ses-1804_task-socword_run-01_eegMRbvCBbviiR250.vhdr', '/Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1804/roi/sub-101472_ses-1804_task-socword_run-01.csv')]['eeg']

KeyError: ('/Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1804/eeg/sub-101472_ses-1804_task-socword_run-01_eegMRbvCBbviiR250.vhdr', '/Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1804/roi/sub-101472_ses-1804_task-socword_run-01.csv')

In [70]:
datasets[('/Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/eeg/sub-101472_ses-1818_task-socword_run-01_eegMRbvCBbviiR250.vhdr', '/Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/roi/sub-101472_ses-1818_task-socword_run-01.csv')]['eeg'].shape

(45, 127, 1250)

In [508]:
root = '/Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472'
sessions = ['1804', '1818', '1895']
channels_dict_1804= {}
channels_dict_1818= {}
channels_dict_1895= {}

for task, run  in RUNS: 
    filename_1804= f'sub-101472_ses-{sessions[0]}_task-{task}_run-{run}_eegMRbvCBbviiR250.vhdr'
    filename_1818= f'sub-101472_ses-{sessions[1]}_task-{task}_run-{run}_eegMRbvCBbviiR250.vhdr'

    eeg_vhdr_path_1804 = os.path.join(root, f'ses-{sessions[0]}', 'eeg', filename_1804)
    eeg_vhdr_path_1818 = os.path.join(root, f'ses-{sessions[1]}', 'eeg', filename_1818)

    eeg_raw_1804 = mne.io.read_raw_brainvision(eeg_vhdr_path_1804, preload = True)
    eeg_raw_1818 = mne.io.read_raw_brainvision(eeg_vhdr_path_1818, preload = True)

    df_eeg_1804 = eeg_raw_1804.to_data_frame()
    df_eeg_1818 = eeg_raw_1818.to_data_frame()

    channels_dict_1804[(task, run)] = df_eeg_1804.columns
    channels_dict_1818[(task, run)] = df_eeg_1818.columns


RUNS_1895 = [
    ('rs', '01'),
    ('rs', '02'),
    ('socword', '01'), # FAULTY FILE in eeg ses-1804 and ses-1895
    ('socword', '02'),
    ('socword', '03'),
    ]
for task, run  in RUNS_1895: 
    filename_1895= f'sub-101472_ses-{sessions[2]}_task-{task}_run-{run}_eegMRbvCBbviiR250.vhdr'

    eeg_vhdr_path_1895 = os.path.join(root, f'ses-{sessions[2]}', 'eeg', filename_1895)

    eeg_raw_1895 = mne.io.read_raw_brainvision(eeg_vhdr_path_1895, preload = True)

    df_eeg_1895 = eeg_raw_1895.to_data_frame()

    channels_dict_1895[(task, run)] = df_eeg_1895.columns


    

Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1804/eeg/sub-101472_ses-1804_task-rs_run-01_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138063  =      0.000 ...   552.252 secs...
Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/eeg/sub-101472_ses-1818_task-rs_run-01_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138060  =      0.000 ...   552.240 secs...


/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_31679/1459616449.py:14: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw_1804 = mne.io.read_raw_brainvision(eeg_vhdr_path_1804, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_31679/1459616449.py:14: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw_1804 = mne.io.read_raw_brainvision(eeg_vhdr_path_1804, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_31679/1459616449.py:15: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw_1818 = mn

Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1804/eeg/sub-101472_ses-1804_task-rs_run-02_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138062  =      0.000 ...   552.248 secs...
Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/eeg/sub-101472_ses-1818_task-rs_run-02_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138059  =      0.000 ...   552.236 secs...


/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_31679/1459616449.py:14: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw_1804 = mne.io.read_raw_brainvision(eeg_vhdr_path_1804, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_31679/1459616449.py:14: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw_1804 = mne.io.read_raw_brainvision(eeg_vhdr_path_1804, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_31679/1459616449.py:15: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw_1818 = mn

Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1804/eeg/sub-101472_ses-1804_task-rs_run-03_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138068  =      0.000 ...   552.272 secs...
Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/eeg/sub-101472_ses-1818_task-rs_run-03_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138055  =      0.000 ...   552.220 secs...


...

/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_31679/1459616449.py:36: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw_1895 = mne.io.read_raw_brainvision(eeg_vhdr_path_1895, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_31679/1459616449.py:36: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw_1895 = mne.io.read_raw_brainvision(eeg_vhdr_path_1895, preload = True)


Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1895/eeg/sub-101472_ses-1895_task-socword_run-01_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 0  =      0.000 ...     0.000 secs...
Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1895/eeg/sub-101472_ses-1895_task-socword_run-02_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138057  =      0.000 ...   552.228 secs...


/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_31679/1459616449.py:36: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw_1895 = mne.io.read_raw_brainvision(eeg_vhdr_path_1895, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_31679/1459616449.py:36: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw_1895 = mne.io.read_raw_brainvision(eeg_vhdr_path_1895, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_31679/1459616449.py:36: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw_1895 = mn

Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1895/eeg/sub-101472_ses-1895_task-socword_run-03_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138024  =      0.000 ...   552.096 secs...


/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_31679/1459616449.py:36: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw_1895 = mne.io.read_raw_brainvision(eeg_vhdr_path_1895, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_31679/1459616449.py:36: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw_1895 = mne.io.read_raw_brainvision(eeg_vhdr_path_1895, preload = True)


In [549]:
for key, single_dataset in datasets_1818.items(): 
    print(key, single_dataset)

('rs', '01') {'eeg': array([[-24.79164314, -27.66599083, -29.34505463, ...,  27.45814514,
         23.09992218,  23.99835205],
       [ 74.06660461,  73.18362427,  72.87739563, ...,  43.7720108 ,
         37.43661499,  37.40782547],
       [-27.6242733 , -26.96910286, -24.72810364, ..., -49.82534027,
        -54.91505814, -54.95967484],
       ...,
       [149.39781189, 172.56297302, 182.53038025, ...,  47.81315613,
         49.40104294,  58.15747452],
       [-28.59513664, -10.63874054,   5.69535351, ..., -13.38128853,
        -34.61465454, -22.88451576],
       [ 65.75875092,  69.30219269,  71.70806122, ...,  37.52774429,
         37.21931076,  36.84614182]]), 'fmri': array([[ 5.67655400e+03, -5.06647369e+01, -1.24420869e+01, ...,
        -5.91139569e+01, -4.91764569e+01, -6.36256769e+01],
       [ 1.18274935e+04, -6.84752602e+01, -4.92955702e+01, ...,
        -7.72916702e+01, -7.42604202e+01, -8.05924502e+01],
       [ 4.10684121e+04, -3.05619172e+02, -2.68494172e+02, ...,
        -

In [252]:
sub_101804_ses_01_task_rs_run_01

{'eeg': array([[-5.23262825e+01, -5.71585045e+01, -5.55627937e+01, ...,
          1.40535385e+02,  1.42954086e+02,  1.35589264e+02],
        [ 1.14316093e+02,  1.22708542e+02,  1.21828667e+02, ...,
          4.36576691e+01,  3.44242592e+01,  3.17797756e+01],
        [-1.36810076e+00, -4.28740978e+00, -3.92652154e+00, ...,
          1.06592323e+02,  1.11197403e+02,  1.11151009e+02],
        ...,
        [ 2.84514313e+02,  2.92167450e+02,  2.94529541e+02, ...,
          1.83335075e+01,  2.34661961e+01,  2.53440876e+01],
        [ 9.74740829e+01,  9.76128769e+01,  9.58119507e+01, ...,
          1.27545532e+02,  1.35939026e+02,  1.52988831e+02],
        [ 1.27199471e-04,  1.27199471e-04,  1.31214615e-04, ...,
         -1.56012527e-04, -1.87589035e-04, -2.10312866e-04]]),
 'fmri': array([[ 7.63077830e+03,  4.45727112e+03,  2.22386357e+03, ...,
          2.28572420e+01,  1.74533397e+01,  5.15722837e+00],
        [ 1.40507030e+04,  8.23360966e+03,  4.13332966e+03, ...,
         -3.18236142e+0

In [69]:

datasets = {}
datasets_nat = {}
all_segment_scans = {}
all_unique_scan_diffs = {}

for ses, task in RUNS:
    df_eeg, eeg_channels, eeg_path, eeg_times, events, event_ids, segment_scans, counts_diffs, unique_scan_diffs = load_eeg_data(
        root= '/Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/intermittentDataset', sub = '101804', task=task, eeg_channels=None)
    #df_fmri_interp, rois, fmri_path, fmri_required_times, fmri_times, fmri_array = load_fmri_data(
        #root = '/Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/intermittentDataset', sub = '101804', ses = '01', task = task, 
   # )
    eeg_array = df_eeg.to_numpy().T

    datasets[eeg_path] = {'eeg': eeg_array, 'df_eeg' : df_eeg, 'eeg_times': eeg_times,'task': task.split('_')[0], 'event_ids':event_ids, 'events':events}
    all_segment_scans[f'{ses}, {task}'] = segment_scans
    all_unique_scan_diffs[f'{ses}, {task}'] = unique_scan_diffs


# for ses, task in RUNSNATVIEW:
#     df_eeg, eeg_channels, eeg_path, eeg_times, events, event_ids = load_natdataset_eeg_data(
#         root= '/Users/terekhova/Desktop/EEG-BOLD-Decoding/NaturalViewingDataset', sub = '04',ses = '01', task=task, eeg_channels=None)
#     eeg_array = df_eeg.to_numpy().T

#     datasets_nat[eeg_path] = {'eeg': eeg_array, 'df_eeg' : df_eeg, 'eeg_times': eeg_times,'task': task.split('_')[0], 'event_ids':event_ids, 'events':events} 


Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/intermittentDataset/sub-101804/ses-01/eeg/sub-101804_ses-01_task-rs_run-01_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138063  =      0.000 ...   552.252 secs...
Used Annotations descriptions: ['New Segment/', 'Pulse Artifact/R', 'Scanner/Scan Start']
Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/intermittentDataset/sub-101804/ses-01/eeg/sub-101804_ses-01_task-rs_run-02_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138062  =      0.000 ...   552.248 secs...


/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_31679/1142039959.py:4: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_31679/1142039959.py:4: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_31679/1142039959.py:4: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw = mne.io.read_raw_brainvision(ee

Used Annotations descriptions: ['New Segment/', 'Pulse Artifact/R', 'Scanner/Scan Start']
Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/intermittentDataset/sub-101804/ses-01/eeg/sub-101804_ses-01_task-rs_run-03_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138068  =      0.000 ...   552.272 secs...
Used Annotations descriptions: ['New Segment/', 'Pulse Artifact/R', 'Scanner/Scan Start']
Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/intermittentDataset/sub-101804/ses-01/eeg/sub-101804_ses-01_task-rs_run-04_eegMRbvCBbviiR250.vhdr...


/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_31679/1142039959.py:4: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_31679/1142039959.py:4: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_31679/1142039959.py:4: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling ins

Setting channel info structure...
Reading 0 ... 138058  =      0.000 ...   552.232 secs...
Used Annotations descriptions: ['New Segment/', 'Pulse Artifact/R', 'Scanner/Scan Start']
Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/intermittentDataset/sub-101804/ses-01/eeg/sub-101804_ses-01_task-rs_run-05_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138057  =      0.000 ...   552.228 secs...
Used Annotations descriptions: ['New Segment/', 'Pulse Artifact/R', 'Scanner/Scan Start']


...

/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_31679/1142039959.py:4: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_31679/1142039959.py:4: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_31679/1142039959.py:4: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw = mne.io.read_raw_brainvision(ee

Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/intermittentDataset/sub-101804/ses-01/eeg/sub-101804_ses-01_task-socword_run-04_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138056  =      0.000 ...   552.224 secs...
Used Annotations descriptions: ['New Segment/', 'Pulse Artifact/R', 'Scanner/Scan Start', 'Stimulus/S  1', 'Stimulus/S  3', 'Stimulus/S  5', 'Stimulus/S  9', 'Stimulus/S 10', 'Stimulus/S 16', 'Stimulus/S 32', 'Stimulus/S 48', 'Stimulus/S 64', 'Stimulus/S 80', 'Stimulus/S 96']
Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/intermittentDataset/sub-101804/ses-01/eeg/sub-101804_ses-01_task-socword_run-05_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138058  =      0.000 ...   552.232 secs...
Used Annotations descriptions: ['New Segment/', 'Pulse Artifact/R', 'Scanner/Scan Start', 'Stimulus/S  1', 'Stimulus/S  3', 'Stimulus/S  5', 'Stimulus/S  9', 'Stim

/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_31679/1142039959.py:4: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_31679/1142039959.py:4: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_31679/1142039959.py:4: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw = mne.io.read_raw_brainvision(ee

Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/intermittentDataset/sub-101804/ses-01/eeg/sub-101804_ses-01_task-socword_run-06_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138054  =      0.000 ...   552.216 secs...
Used Annotations descriptions: ['New Segment/', 'Pulse Artifact/R', 'Scanner/Scan Start', 'Stimulus/S  1', 'Stimulus/S  3', 'Stimulus/S  5', 'Stimulus/S  9', 'Stimulus/S 10', 'Stimulus/S 32', 'Stimulus/S 48', 'Stimulus/S 64', 'Stimulus/S 80', 'Stimulus/S 96']


/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_31679/1142039959.py:4: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_31679/1142039959.py:4: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)


In [40]:
np.unique(eeg_01['events'][:, -1])

array([10001, 10002, 99999])

In [275]:
events = eeg_01['events']
event_ids = eeg_01['event_ids']
scan_events = events[events[:, -1] == event_ids['Scanner/Scan Start']]

# eeg_02 == event_ids['Scanner/Scan Start']

In [278]:
len(scan_events)

46

In [750]:
events[-30:]

array([[132776,      0,  10001],
       [132946,      0,  10001],
       [133118,      0,  10001],
       [133291,      0,  10001],
       [133468,      0,  10001],
       [133653,      0,  10001],
       [133846,      0,  10001],
       [134041,      0,  10001],
       [134232,      0,  10001],
       [134423,      0,  10001],
       [134614,      0,  10001],
       [134804,      0,  10001],
       [134987,      0,  10001],
       [135032,      0,  10002],
       [135168,      0,  10001],
       [135349,      0,  10001],
       [135529,      0,  10001],
       [135709,      0,  10001],
       [135889,      0,  10001],
       [136072,      0,  10001],
       [136257,      0,  10001],
       [136449,      0,  10001],
       [136634,      0,  10001],
       [136815,      0,  10001],
       [136995,      0,  10001],
       [137177,      0,  10001],
       [137360,      0,  10001],
       [137535,      0,  10001],
       [137712,      0,  10001],
       [137895,      0,  10001]])

In [745]:
events[events[:,2] == 10002].shape

(46, 3)

In [22]:

root = '/Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472'
eeg_channels = None
sampling_rates_ratio = 500
leeway = 0.001
filename = f'sub-101472_ses-1818_task-socword_run-06_eegMRbvCBbviiR250.vhdr'
eeg_vhdr_path = os.path.join(root, f'ses-1818', 'eeg', filename)
eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)

events, event_ids = mne.events_from_annotations(eeg_raw)
events_dict_cut = {'Scanner/Scan Start': event_ids['Scanner/Scan Start']}  # use only Scan Start event



try:
    scan_key = 'Scanner/Scan Start'
    segment_scans = events[events[:, -1] == event_ids[scan_key]]
except: 
    scan_key = 'Stimulus/Scan Start'
    segment_scans = events[events[:, -1] == event_ids[scan_key]] 
    print(f'ERROR: FAULTY EVENT NAME. PATH:{eeg_vhdr_path}')


unique_scan_diffs, counts_diffs = np.unique(np.diff(segment_scans[:, 0]), return_counts=True)

first_segment = segment_scans[0, 0] / EEG_SAMPLING_RATE
cropped_eeg = eeg_raw.crop(tmin=first_segment)

df_eeg = eeg_raw.to_data_frame()
cropped_df_eeg = cropped_eeg.to_data_frame()

if eeg_channels is None:
    eeg_channels = [column for column in df_eeg.columns if column != 'time']

eeg_times = df_eeg['time']
cropped_eeg_times = cropped_df_eeg['time']

df_eeg = df_eeg[eeg_channels]


fmri_array, fmri_times, rois, fmri_path, df_fmri = load_fmri_data(
        root=root, run='01', ses='1818', task='rs', rois = None, roi_folder='roi'
    )





n_eeg_samples = df_fmri.shape[0] * sampling_rates_ratio
df_eeg_new = df_eeg.iloc[:n_eeg_samples]
eeg_array = df_eeg_new.to_numpy().T

epochs = mne.Epochs(
        cropped_eeg,
        events = segment_scans, 
        event_id =events_dict_cut, 
        tmin = 6.5,
        tmax = 11.5-1/EEG_SAMPLING_RATE, 
        baseline = None, 
        preload = True
    )
print(epochs.drop_log)

epochs_df = epochs.to_data_frame()
#epochs_df = epochs_df[:-1]
segment_scans = segment_scans[:-1]

#epochs = epochs.copy().filter(1, 70)
#epochs = epochs.crop(tmin=6 + leeway, tmax=12 - leeway, include_tmax=False)


Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/eeg/sub-101472_ses-1818_task-socword_run-06_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138056  =      0.000 ...   552.224 secs...
Used Annotations descriptions: ['New Segment/', 'Scanner/Scan Start', 'Stimulus/S  1', 'Stimulus/S  3', 'Stimulus/S  5', 'Stimulus/S  9', 'Stimulus/S 10', 'Stimulus/S 16', 'Stimulus/S 32', 'Stimulus/S 48', 'Stimulus/S 64', 'Stimulus/S 80', 'Stimulus/S 96']


/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_75595/333210420.py:7: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_75595/333210420.py:7: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)


Not setting metadata
46 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 46 events and 1250 original time points ...
0 bad epochs dropped
((), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), ())


In [33]:
epochs_df.to_numpy().shape

(57500, 130)

In [822]:
# assert for tsart integer 
all([i.is_integer() for i in df_fmri[' Tstart']])
fmri_times = fmri_times.astype(int)
fmri_times

array([  0,   2,   4,  12,  14,  16,  24,  26,  28,  36,  38,  40,  48,
        50,  52,  60,  62,  64,  72,  74,  76,  84,  86,  88,  96,  98,
       100, 108, 110, 112, 120, 122, 124, 132, 134, 136, 144, 146, 148,
       156, 158, 160, 168, 170, 172, 180, 182, 184, 192, 194, 196, 204,
       206, 208, 216, 218, 220, 228, 230, 232, 240, 242, 244, 252, 254,
       256, 264, 266, 268, 276, 278, 280, 288, 290, 292, 300, 302, 304,
       312, 314, 316, 324, 326, 328, 336, 338, 340, 348, 350, 352, 360,
       362, 364, 372, 374, 376, 384, 386, 388, 396, 398, 400, 408, 410,
       412, 420, 422, 424, 432, 434, 436, 444, 446, 448, 456, 458, 460,
       468, 470, 472, 480, 482, 484, 492, 494, 496, 504, 506, 508, 516,
       518, 520, 528, 530, 532, 540, 542, 544])

In [862]:
fmri_times = fmri_times//12
fmri_times.shape

(138,)

In [826]:

df_fmri = df_fmri.assign(N_cycle=pd.Series(fmri_times).values)

In [833]:
df_ncycle = df_fmri.groupby(by = 'N_cycle')
len(df_ncycle)

46

In [834]:
rois

[' Left Cerebral White Matter',
 ' Left Cerebral Cortex',
 ' Left Lateral Ventricle',
 ' Left Thalamus',
 ' Left Caudate',
 ' Left Putamen',
 ' Left Pallidum',
 ' Left Brain-Stem',
 ' Right Brain-Stem',
 ' Left Hippocampus',
 ' Left Amygdala',
 ' Left Accumbens',
 ' Right Cerebral White Matter',
 ' Right Cerebral Cortex',
 ' Right Lateral Ventricle',
 ' Right Thalamus',
 ' Right Caudate',
 ' Right Putamen',
 ' Right Pallidum',
 ' Right Hippocampus',
 ' Right Amygdala',
 ' Right Accumbens']

In [ ]:
fmri_array = np.full((len(df_ncycle), len(rois), 3), fill_value = np.inf)

In [842]:
for idx, group in df_ncycle: 
    group = group[rois].to_numpy()
    fmri_array[idx, ...] = group.T 

In [845]:
# check if no inf that were previously filled are left  
assert not np.isinf(fmri_array).any()  ,(f'')  

In [847]:
fmri_array.shape

(46, 22, 3)

In [790]:
print(epochs)

<Epochs | 46 events (all good), 6 – 11.996 s (baseline off), ~67.0 MB, data loaded,
 'Scanner/Scan Start': 46>


In [791]:
for idx, epoch in enumerate(epochs): 
    print(idx, epoch.shape)

0 (127, 1500)
1 (127, 1500)
2 (127, 1500)
3 (127, 1500)
4 (127, 1500)
5 (127, 1500)
6 (127, 1500)
7 (127, 1500)
8 (127, 1500)
9 (127, 1500)
10 (127, 1500)
11 (127, 1500)
12 (127, 1500)
13 (127, 1500)
14 (127, 1500)
15 (127, 1500)
16 (127, 1500)
17 (127, 1500)
18 (127, 1500)
19 (127, 1500)
20 (127, 1500)
21 (127, 1500)
22 (127, 1500)
23 (127, 1500)
24 (127, 1500)
25 (127, 1500)
26 (127, 1500)
27 (127, 1500)
28 (127, 1500)
29 (127, 1500)
30 (127, 1500)
31 (127, 1500)
32 (127, 1500)
33 (127, 1500)
34 (127, 1500)
35 (127, 1500)
36 (127, 1500)
37 (127, 1500)
38 (127, 1500)
39 (127, 1500)
40 (127, 1500)
41 (127, 1500)
42 (127, 1500)
43 (127, 1500)
44 (127, 1500)
45 (127, 1500)


In [792]:
print(epochs[:-1])

<Epochs | 45 events (all good), 6 – 11.996 s (baseline off), ~65.6 MB, data loaded,
 'Scanner/Scan Start': 45>


In [783]:
segment_scans

array([[    32,      0,  10002],
       [  3032,      0,  10002],
       [  6032,      0,  10002],
       [  9032,      0,  10002],
       [ 12032,      0,  10002],
       [ 15032,      0,  10002],
       [ 18032,      0,  10002],
       [ 21032,      0,  10002],
       [ 24032,      0,  10002],
       [ 27032,      0,  10002],
       [ 30032,      0,  10002],
       [ 33032,      0,  10002],
       [ 36032,      0,  10002],
       [ 39032,      0,  10002],
       [ 42032,      0,  10002],
       [ 45032,      0,  10002],
       [ 48032,      0,  10002],
       [ 51032,      0,  10002],
       [ 54032,      0,  10002],
       [ 57032,      0,  10002],
       [ 60032,      0,  10002],
       [ 63032,      0,  10002],
       [ 66032,      0,  10002],
       [ 69032,      0,  10002],
       [ 72032,      0,  10002],
       [ 75032,      0,  10002],
       [ 78032,      0,  10002],
       [ 81032,      0,  10002],
       [ 84032,      0,  10002],
       [ 87032,      0,  10002],
       [ 9

In [894]:
epochs_df['epoch'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45])

In [915]:
events

array([[     0,      0,  99999],
       [    29,      0,  10001],
       [  3029,      0,  10001],
       [  6029,      0,  10001],
       [  9029,      0,  10001],
       [ 12029,      0,  10001],
       [ 15029,      0,  10001],
       [ 18029,      0,  10001],
       [ 21029,      0,  10001],
       [ 24029,      0,  10001],
       [ 27029,      0,  10001],
       [ 30029,      0,  10001],
       [ 33029,      0,  10001],
       [ 36029,      0,  10001],
       [ 39029,      0,  10001],
       [ 42029,      0,  10001],
       [ 45029,      0,  10001],
       [ 48029,      0,  10001],
       [ 51029,      0,  10001],
       [ 54029,      0,  10001],
       [ 57029,      0,  10001],
       [ 60029,      0,  10001],
       [ 63029,      0,  10001],
       [ 66029,      0,  10001],
       [ 69029,      0,  10001],
       [ 72029,      0,  10001],
       [ 75029,      0,  10001],
       [ 78029,      0,  10001],
       [ 81029,      0,  10001],
       [ 84029,      0,  10001],
       [ 8

In [906]:
event_ids

{'New Segment/': 99999, 'Pulse Artifact/R': 10001, 'Scanner/Scan Start': 10002}

In [34]:

(epochs_df['epoch'].unique() == np.where(events[:,2] == 10002)).all()

ValueError: operands could not be broadcast together with shapes (46,) (1,0) 

In [794]:
np.where(events[:,2] == 10002)

(array([  1,  17,  34,  51,  68,  84, 101, 119, 136, 153, 169, 186, 203,
        220, 237, 254, 271, 287, 305, 322, 339, 356, 374, 392, 410, 427,
        445, 461, 478, 494, 510, 526, 543, 559, 576, 592, 608, 625, 642,
        659, 677, 694, 711, 729, 746, 764]),)

In [781]:
epochs_df.head()

,time,condition,epoch,Fp1,Fp2,F3,F4,C3,C4,P3,...,F9,F10,P9,P10,PO9,PO10,O9,O10,Iz,CPz
0,6.000,Scanner/Scan Start,1,18.120977,24.699791,-34.733421,29.073410,38.395153,52.658619,-9.866400,...,24.603662,82.468216,49.363777,36.200348,135.436234,56.470100,8.761945,69.182129,175.501511,55.956303
1,6.004,Scanner/Scan Start,1,17.754728,29.786974,-32.858784,32.884209,39.696404,55.524582,-8.915770,...,22.289364,90.127335,44.174641,40.959824,114.598022,33.235401,-13.482180,34.650360,149.575073,57.100334
2,6.008,Scanner/Scan Start,1,21.268997,30.953688,-31.629755,32.699211,37.685951,54.211086,-5.763182,...,-156.959686,93.477341,36.823471,48.110039,79.290382,3.047441,-42.835194,-40.277081,123.500191,56.663879
3,6.012,Scanner/Scan Start,1,22.171379,31.584776,-29.957188,31.436586,36.927265,53.447399,-6.084198,...,-304.660461,92.301437,48.778217,46.657284,77.000664,6.329995,-38.838932,-74.777245,128.490036,57.675632
4,6.016,Scanner/Scan Start,1,22.235443,28.112070,-35.170029,27.697300,36.366440,51.642910,-4.496112,...,-41.074226,89.514679,52.840595,45.098473,106.853355,34.730602,-12.794170,-68.273857,155.114914,56.500381


In [758]:
print(epochs.drop_log)

(('IGNORED',), (), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), (), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), (), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), (), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), (), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',),

In [40]:
epochs_df.to_numpy().shape

(57500, 130)

In [155]:
epochs.get_data()[:-1, ...].shape

(45, 127, 1250)

In [156]:
from sklearn.model_selection import train_test_split

In [157]:
def split_single_dataset(eeg: np.ndarray, fmri: np.ndarray, task: str): 
    
    # зафиксировать by % test and val, put the rest in train
    train_size = 25
    val_size = 10
    test_size = 10
    
    eeg_train, eeg_test, fmri_train, fmri_test = train_test_split(
        eeg, fmri, test_size=0.25, shuffle=False
        )

    eeg_train, eeg_val, fmri_train, fmri_val = train_test_split(
        eeg_train, fmri_train, test_size=0.2, shuffle=False
        ) 

    train_data = {'eeg': eeg_train,
                  'fmri': fmri_train}

    val_data = {'eeg': eeg_val,
                'fmri': fmri_val}

    test_data = {'eeg': eeg_test,
                 'fmri': fmri_test}

    return train_data, val_data, test_data

def split_datasets(datasets: dict):
    train_datasets = {}
    validation_datasets = {}
    test_datasets = {}

    for set_path, single_dataset in datasets.items():

        train_data, val_data, test_data = split_single_dataset(
            eeg=single_dataset['eeg'], fmri=single_dataset['fmri'],
            task=single_dataset['task']
        )

        train_datasets[set_path] = train_data

        if val_data is not None:
            validation_datasets[set_path] = val_data

        if test_data is not None:
            test_datasets[set_path] = test_data

    return train_datasets, validation_datasets, test_datasets




In [158]:
train_datasets, validation_datasets, test_datasets = split_datasets(datasets)

In [217]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, ConcatDataset, RandomSampler

In [175]:
# initialise for each run and then concatDataset <- rewrite from build_full_dataset 
class EEGFMRIDataset(Dataset):
    def __init__(self, eeg, fmri, data_path, eeg_channels, fmri_rois):
        self.data_path = data_path
        self.eeg = torch.from_numpy(eeg).float()
        self.fmri = torch.from_numpy(fmri).float()

        self.eeg_channels = eeg_channels
        self.fmri_rois = list(fmri_rois)

        assert self.eeg.size(0) == self.fmri.size(0), (self.eeg.size(), self.fmri.size())

    def __getitem__(self, idx):
        return self.eeg[idx, ...], self.fmri[idx, ...]

    def __len__(self):
        return self.eeg.size(0)
    
    def get_rois(self):
        return list(self.fmri_rois)
    
def build_full_dataset(datasets:dict, fmri_rois:list[str]):
    torch_datasets = []
    all_rois = None
    for ds_path, dataset in datasets.items():
        torch_dataset = EEGFMRIDataset(
            eeg=dataset['eeg'], fmri=dataset['fmri'],
            eeg_channels=eeg_channels, fmri_rois=fmri_rois,
            data_path=ds_path)
        torch_rois = torch_dataset.get_rois()
        if all_rois is None:
            all_rois = list(torch_rois)
        else:
            assert all_rois == torch_rois, (all_rois, torch_rois)
        torch_datasets.append(torch_dataset)
    full_dataset = ConcatDataset(torch_datasets)

    return full_dataset, all_rois


In [ ]:
torch_datasets = {}
all_datasets = {'train': train_datasets, 'validation': validation_datasets, 'test': test_datasets}

for split_name in ['train', 'validation', 'test']:

    split_dataset, split_rois = build_full_dataset(
        datasets=all_datasets[split_name], fmri_rois=rois)

    torch_datasets[split_name] = {'dataset': split_dataset, 'rois': split_rois}

In [179]:
train_dataset, validation_dataset, test_dataset = torch_datasets['train']['dataset'], torch_datasets['validation']['dataset'], torch_datasets['test']['dataset']

In [198]:

from typing import Optional, Dict, Any, Tuple


In [ ]:

class BrainWaveLinker(nn.Module):
    def __init__(self,
                 nn_parameters: dict,
                 roi_names: list[str], 
                 eeg_channel_names: list[str],
                 criterion_name: str, 
                 criterion_kwargs: dict,
                 optimizer_name: str, 
                 optimizer_kwargs: dict,
                 scheduler_kwargs: dict ):
        
        super(BrainWaveLinker, self).__init__()

        # Initialize model parameters
        in_channels = len(eeg_channel_names)
        out_channels = len(roi_names)

        # Model architecture
        intermediate_channels = nn_parameters.get('intermediate_channels', out_channels)
        if intermediate_channels is None:
            intermediate_channels = out_channels

        self.spatial_filter = nn.Conv1d(in_channels, intermediate_channels, kernel_size=1)

        self.pyramidal_subsampling = nn.Sequential(
            nn.GELU(),
            nn.Conv1d(in_channels=intermediate_channels, out_channels=intermediate_channels, kernel_size=4, stride=4),
            nn.GELU(),
            nn.Dropout(nn_parameters.get('dropout_prob', 0.25)),
            nn.Conv1d(in_channels=intermediate_channels, out_channels=intermediate_channels, kernel_size=4, stride=4),
            nn.GELU(),
            nn.Dropout(nn_parameters.get('dropout_prob', 0.25)),
            nn.Conv1d(in_channels=intermediate_channels, out_channels=intermediate_channels, kernel_size=4, stride=4),
            nn.GELU(),
            nn.Dropout(nn_parameters.get('dropout_prob', 0.25))
        )

        self.temporal_filter = nn.Conv1d(intermediate_channels, out_channels, kernel_size=nn_parameters['temporal_filter_kernel'])

        # Criterion and optimizer setup
        self.criterion = self.build_criterion(criterion_name, criterion_kwargs)
        self.optimizer_name = optimizer_name
        self.optimizer_kwargs = optimizer_kwargs
        


    def build_criterion(self, criterion_name: str, criterion_kwargs: dict):
        def build_criterion(criterion_name: str, criterion_kwargs: dict):
            if criterion_name == 'MSELoss':
                criterion = nn.MSELoss(**criterion_kwargs)
            elif criterion_name == 'L1Loss':
                criterion = nn.MSELoss(**criterion_kwargs)
            else:
                raise NotImplementedError
            return criterion

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        out_sf = self.spatial_filter(inputs)
        out_ps = self.pyramidal_subsampling(out_sf)
        return self.temporal_filter(out_ps)

    def common_step(self, batch: Tuple[torch.Tensor], mode: str) -> Tuple[torch.Tensor]:
        inputs, target = batch
        output = self(inputs)
        
        loss = self.criterion(output, target)
        self.log(f'{mode}_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        
        return loss

    def training_step(self, batch: Tuple[torch.Tensor]):
        return self.common_step(batch=batch, mode='train')

    def validation_step(self, batch: Tuple[torch.Tensor]):
        return self.common_step(batch=batch, mode='val')

    def test_step(self, batch: Tuple[torch.Tensor]):
        return self.common_step(batch=batch, mode='test')

    def configure_optimizers(self):
        optimizers = {
            'Adam': torch.optim.Adam,
            'AdamW': torch.optim.AdamW,
            'RMSprop': torch.optim.RMSprop,
            'SGD': torch.optim.SGD
        }
        
        optimizer = optimizers[self.optimizer_name](self.parameters(), **self.optimizer_kwargs)
        
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, **scheduler_kwargs)
        
        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': scheduler,
                'monitor': 'val_loss'
            },
        }




In [213]:
model_config = {"temporal_filter_kernel": 12, "dropout_prob": 0.25, "intermediate_channels": None}
scheduler_config = {"mode": "min", "factor": 0.1, "patience": 5}

TypeError: 'BrainWaveLinker' object is not subscriptable

In [ ]:
brain_wave_linker_net = BrainWaveLinker(
        nn_parameters=model_config,
        eeg_channel_names=eeg_channels,
        roi_names=rois,
        criterion_name='MSELoss',
        criterion_kwargs={"dim": -1, "eps": 1e-06},
        optimizer_name='Adam',
        optimizer_kwargs={"lr": 0.001,"weight_decay": 0.001},
        scheduler_kwargs=scheduler_config
    )

In [ ]:
# 0.5 create git repository 
#  1. переписать bw_linker под окно размера 6s на входе  
# temporal_filter_kernel = 12 
# 2. for 3 datapoints kernel_size & stride == 4 
    # alternatively try temporal_filter_kernel = 10 
# 3. consider writing in torch instead of lightning 
    # in case of choosing lightning, rewrite metrics calculation
# 4. loss should be mse instead of neg_corr 

In [218]:
def build_dataloader(dataset: torch.utils.data.Dataset, sampler_parameters: Optional[dict],
                     dataloader_parameters: dict):

    samplers = {
        'RandomSampler': RandomSampler
    }

    if sampler_parameters is not None:
        sampler = samplers[sampler_parameters['name']](dataset, **sampler_parameters['parameters'])
    else:
        sampler = None

    dataloader = torch.utils.data.DataLoader(dataset, sampler=sampler, **dataloader_parameters)

    return dataloader


In [216]:

brain_wave_linker_net

BrainWaveLinker(
  (spatial_filter): Conv1d(127, 22, kernel_size=(1,), stride=(1,))
  (pyramidal_subsampling): Sequential(
    (0): GELU(approximate='none')
    (1): Conv1d(22, 22, kernel_size=(4,), stride=(4,))
    (2): GELU(approximate='none')
    (3): Dropout(p=0.25, inplace=False)
    (4): Conv1d(22, 22, kernel_size=(4,), stride=(4,))
    (5): GELU(approximate='none')
    (6): Dropout(p=0.25, inplace=False)
    (7): Conv1d(22, 22, kernel_size=(4,), stride=(4,))
    (8): GELU(approximate='none')
    (9): Dropout(p=0.25, inplace=False)
  )
  (temporal_filter): Conv1d(22, 22, kernel_size=(12,), stride=(1,))
)

(26, 22, 3)

In [247]:
dataloader_configs = {"train": {"batch_size": 128,"drop_last": False,"num_workers": 0},  "validation": {
        "batch_size": 128,
        "drop_last": False,
        "num_workers": 0
    },
    "test": {
        "batch_size": 128,
        "drop_last": False,
        "num_workers": 0
    }
}

sampler_configs = {
        "train": None,
        "validation": None
    }

In [ ]:
train_loader = build_dataloader(dataset=train_dataset, sampler_parameters=sampler_configs['train'],
                                    dataloader_parameters=dataloader_configs['train'])
val_loader = build_dataloader(dataset=validation_dataset, sampler_parameters=sampler_configs['validation'],
                                  dataloader_parameters=dataloader_configs['validation'])
test_loader = build_dataloader(dataset=test_dataset, sampler_parameters=None,
                                   dataloader_parameters=dataloader_configs['test'])

In [231]:
print('train_loader', len(train_loader))
print('val_loader', len(val_loader))
print('test_loader', len(test_loader))

train_loader 4
val_loader 1
test_loader 2


In [234]:
trainer_config= {
        "early_stopping": {
            "monitor": "val_loss",
            "mode": "min",
            "patience": 20
        },
        "checkpointing": {
            "monitor": "val_loss",
            "mode": "min",
            "save_top_k": 1
        },
        "kwargs": {
            "max_epochs": 250,
            "log_every_n_steps": 1
        }
    }

In [248]:
import torch

from torch.utils.data import DataLoader


# Move model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
brain_wave_linker_net.to(device)

# Define the optimizer
learning_rate = 0.001  # Define this in your trainer_config
optimizer = torch.optim.Adam(brain_wave_linker_net.parameters(), lr= 0.001,weight_decay=0.001)

# Optionally, define a learning rate scheduler if needed
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, **scheduler_config)

# Training loop parameters
num_epochs = 50  # Number of epochs from trainer_config

# Variable to store the best model state in memory
best_model_state = None
best_loss = float('inf')

criterion_kwargs= {"dim": -1, "eps":1e-06}
criterion = nn.MSELoss()
# Training and validation loop
for epoch in range(num_epochs):
    # Training phase
    brain_wave_linker_net.train()
    total_loss = 0.0
    
    for batch in train_loader:
        inputs, targets = batch
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()  # Clear gradients
        outputs = brain_wave_linker_net(inputs)  # Forward pass
        loss = criterion(outputs, targets)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}')

    # Validation phase
    brain_wave_linker_net.eval()
    total_val_loss = 0.0

    with torch.no_grad():
        for batch in val_loader:
            inputs, targets = batch
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = brain_wave_linker_net(inputs)
            loss = criterion(outputs, targets)
            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(val_loader)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f}')



    # Save the best model state based on validation loss in memory
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        best_model_state = brain_wave_linker_net.state_dict()  # Save state dict in memory

print('Training complete. Best model state is stored in memory.')

# Testing phase using the best model state stored in memory
brain_wave_linker_net.load_state_dict(best_model_state)  # Load the best model state from memory
brain_wave_linker_net.eval()

test_loader = DataLoader(test_dataset, batch_size=trainer_config['batch_size'], shuffle=False)

with torch.no_grad():
    for batch in test_loader:
        inputs, targets = batch
        inputs = inputs.to(device)
        
        outputs = brain_wave_linker_net(inputs)
        # Process outputs as needed for your application




Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/terekhova/fsl/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/terekhova/fsl/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/terekhova/fsl/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/terekhova/fsl/lib/python3.12/multiprocessing/spawn.py", line 132, in _main
    exitcode = _main(fd, parent_sentinel)
    exitcode = _main(fd, parent_sentinel)Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/terekhova/fsl/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main

               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/terekhova

RuntimeError: DataLoader worker (pid(s) 89908) exited unexpectedly

In [ ]:

from bw_linker.brain_wave_pipeline.brain_wave_linker import BrainWaveLinkerSystem
from bw_linker.data_preprocessing.load_brain_data import load_multiple_files, split_datasets
from bw_linker.utils.constants import EEG_SAMPLING_RATE
from bw_linker.utils.helpers import get_run_name, get_window_sizes_sample, print_wrapped_text


In [133]:
from torch.utils.data import ConcatDataset, Dataset
import torch
import math
from scipy.stats import iqr 

from typing import Optional

import lightning as L
import torch.nn as nn



In [ ]:
class EEGFMRIDataset(Dataset): 
    def __init__(self, eeg:np.array, fmri:np.array, fmri_window_size: int, eeg_window_size: int,
                 sampling_rates_ratio: int,
                 eeg_channels: list[str], fmri_rois: list[str],
                 ds_path: Optional[str] = None, stride: int = 1):
        self.eeg = torch.tensor(eeg.astype(np.float32))
        self.fmri = torch.tensor(fmri.astype(np.float32))

        self.sampling_rates_ratio = sampling_rates_ratio  
        self.eeg_channels = eeg_channels
        self.fmri_rois = list(fmri_rois)
        self.ds_path = ds_path 
        self.eeg_window_size = eeg_window_size
        self.fmri_window_size = fmri_window_size
        self.stride = stride 
    def get_item(self, item:int): 
        item *= self.stride 
        fmri_data = self.fmri[:, item:item + self.fmri_window_size]
        eeg_item = item * self.sampling_rates_ratio
        eeg_data = self.eeg[:, eeg_item:eeg_item + self.eeg_window_size]

        assert fmri_data.size(-1) == self.fmri_window_size
        assert eeg_data.size(-1) == self.eeg_window_size
        return eeg_data, fmri_data
    
    def __len__(self):
        return min(
            math.ceil((self.fmri.size(-1) - self.fmri_window_size + 1) / self.stride),
            math.ceil((self.eeg.size(-1) - self.eeg_window_size + 1) / (self.sampling_rates_ratio * self.stride))
        )
    def get_all_data(self):
        return self.eeg, self.fmri, self.ds_path

    def get_rois(self):
        return list(self.fmri_rois)
    
    @staticmethod
    def standardize_data(data: np.array, subtract: Optional[str] = 'mean', divide_by: Optional[str] = 'std',
                         axis: Optional[int] = 0):

        if (subtract is None) and (divide_by is None):
            return data
        functions = {'mean': np.mean, 'median': np.median, 'std': np.std, 'iqr': iqr}
        if subtract is None:
            subtraction_value = 0
        else:
            subtraction_value = functions[subtract](data, axis=axis, keepdims=True)
        if divide_by is None:
            division_value = 1
        else:
            division_value = functions[divide_by](data, axis=axis, keepdims=True)
        return (data - subtraction_value) / division_value

    def build_full_dataset(datasets: dict, fmri_window_size: int, eeg_window_size: int, sampling_rates_ratio: int,
                       eeg_channels: list[str], fmri_rois: list[str], stride: int = 1,
                       eeg_standardization_kwargs: Optional[dict] = None,
                       fmri_standardization_kwargs: Optional[dict] = None):
        torch_datasets = []
        all_rois = None
        for ds_path, dataset in datasets.items():
            torch_dataset = EEGFMRIDataset(
                eeg=dataset['eeg'], fmri=dataset['fmri'],
                fmri_window_size=fmri_window_size, eeg_window_size=eeg_window_size,
                sampling_rates_ratio=sampling_rates_ratio,
                eeg_channels=eeg_channels, fmri_rois=fmri_rois,
                ds_path=ds_path, stride=stride
            )
            torch_rois = torch_dataset.get_rois()
            if all_rois is None:
                all_rois = list(torch_rois)
            else:
                assert all_rois == torch_rois, (all_rois, torch_rois)
            torch_datasets.append(torch_dataset)
        full_dataset = ConcatDataset(torch_datasets)

        return full_dataset, all_rois

        

In [134]:
class NegativeCorrelation(nn.Module):
    """
    An implementation of the Negative Correlation in PyTorch. Intended to be used as a loss function

    Parameters
    ----------
    dim : int
        A dimension over which the correlation is supposed to be computed. Expected input size to module is
        (batch, n_rois, n_times). Default: -1
    eps : float
        Small value to avoid division by 0. Default: 1e-6
    """
    def __init__(self, dim: int = -1, eps: float = 1e-6):
        super().__init__()
        self.cos = nn.CosineSimilarity(dim=dim, eps=eps)
        self.dim = dim

    def forward(self, predictions, targets):
        pearson = self.cos(predictions - predictions.mean(dim=self.dim, keepdim=True),
                           targets - targets.mean(dim=self.dim, keepdim=True))
        return -1. * torch.mean(pearson)


def build_criterion(criterion_name: str, criterion_kwargs: dict):
    """
    Initializes a criterion (loss function) and returns it

    Parameters
    ----------
    criterion_name : str
        A name of the required criterion
    criterion_kwargs : dict
        A dict with keyword arguments for a desired criterion

    Returns
    -------
    criterion : torch.nn.Module
        An initialized criterion
    """
    if criterion_name == 'MSELoss':
        criterion = nn.MSELoss(**criterion_kwargs)
    elif criterion_name == 'L1Loss':
        criterion = nn.MSELoss(**criterion_kwargs)
    elif criterion_name == 'NegativeCorrelation':
        criterion = NegativeCorrelation(**criterion_kwargs)
    else:
        raise NotImplementedError
    return criterion